Preprocessing Data (Milestone 1)

In [90]:
import numpy as np
import tensorflow as tf

In [91]:
def clean(fname):

    # Separate file into books with all metadata
    startBook = "<book"
    books = []
    with open(fname) as openFile:
        for line in openFile:
            if line[:len(startBook)] == startBook:
                books.append(line)
            else:
                books[-1] += line
    openFile.close()

    books = np.array(books)

    # Collect synopses and all genre tags for each book
    # X_data found here (synopses)
    synopses = []
    genres = []

    startBody = "<body>"
    endBody = "</body>"

    startTopic = "<topics>"
    endTopic = "</topics>"

    for book in books:
        start = book.index(startBody) + len(startBody)
        end = book.index(endBody)
        synopses.append(book[start:end])

        start = book.index(startTopic) + len(startTopic)
        end = book.index(endTopic)
        genres.append(book[start:end])

    X_data = np.array(synopses)


    # Create matrix of genre tags to book
    # Y_data found here
    start = ">"
    end = "</"

    Y_data = np.zeros((len(books), len(allGenres)))

    for i in range(len(books[:200])):
        entry = genres[i]
        while entry.find(end) != -1:
            s = entry.index(start) + 1
            e = entry.index(end)
            genreTag = entry[s:e]
            Y_data[i][allGenres.index(genreTag)] = 1
            entry = entry[e+5:]

    return (X_data, Y_data)

In [92]:
# Create a list of all genre tags
allGenres = []
with open("../data/hierarchy.txt") as hierarchyFile:
    for line in hierarchyFile:
        tab = line.find("\t")
        g1 = line[:tab]
        g2 = line[tab+1:len(line)-1]
        if allGenres.count(g1) == 0:
            allGenres.append(g1)
        if allGenres.count(g2) == 0:
            allGenres.append(g2)

print(allGenres[:4])

['Biography & Memoir', 'Arts & Entertainment Biographies & Memoirs', 'Political Figure Biographies & Memoirs', 'Historical Figure Biographies & Memoirs']


In [93]:
X_train, Y_train = clean("../data/BlurbGenreCollection_EN_train.txt")
X_test, Y_test = clean("../data/BlurbGenreCollection_EN_test.txt")
X_valid, Y_valid = clean("../data/BlurbGenreCollection_EN_dev.txt")


print(X_train[0])
print(Y_train[0])

Monday’s Crosswords Do with EaseTuesday’s Crosswords Not a BreezeWednesday’s Crosswords Harder StillThursday’s Crosswords Take Real SkillFriday’s Crosswords — You’ve Come This Far…Saturday’s Crosswords — You’re a Star!For millions of people, the New York Times crossword puzzles are as essential to each day as the first cup of coffee in the morning. Now, for the first time ever, these premier puzzles are available in six clever installments. With each day of the week, the puzzles increase gradually in skill level; Monday’s the easiest, but Saturday’s sure to challenge! Push your mental muscles a little harder each day with America’s favorite sophisticated — and fun — pastime: the New York Times crossword puzzles!The legendary Eugene T. Maleska was crossword editor of The New York Times from 1977 to 1993.
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [94]:
from tensorflow.keras.layers import TextVectorization
import re
import string

In [95]:
# Create Word2Vec word embeddings for each synopsis
# Start by indexing all words that appear into a vocuabulary list

# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and number of words in a sequence.
vocab_size = 10000
seq_length = 400 # approximately the largest synopsis we have

# Use the TextVectorization layer to normalize, split, and map strings to
# integers. Set output_sequence_length length to pad all samples to same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_length)



    

In [96]:
# Create global vocabulary list based off of train data
vectorize_layer.adapt(X_train)
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'of', 'a', 'to', 'in', 'is', 'for', 'with', 'that', 'her', 'his', 'as', 'on', 'from', 'this', 'an', 'by']


In [97]:
# Model to encode synopses to arrays of indices corresponding to words in the global vocab
word2index = tf.keras.models.Sequential()
word2index.add(tf.keras.Input(shape=(1,), dtype=tf.string))
word2index.add(vectorize_layer)

In [98]:
# encoding the synopses
X_train = word2index.predict(X_train)
X_test = word2index.predict(X_test)
X_valid = word2index.predict(X_valid)

In [99]:
for i in range(20, 40):
    print(inverse_vocab[X_train[0][i]])

this
[UNK]
[UNK]
—
you’re
a
[UNK]
millions
of
people
the
new
york
times
crossword
puzzles
are
as
essential
to


In [100]:
vocab_2_index = dict((c,i) for i, c in enumerate(inverse_vocab)) 
index_2_vocab = dict((i,c) for i, c in enumerate(inverse_vocab))

In [101]:
import pandas as pd

In [102]:
metadata = pd.read_csv(open('metadata.tsv'), header=None)

In [103]:
metadata.head(10)

,0
0,[UNK]
1,the
2,and
3,of
4,a
5,to
6,in
7,is
8,for
9,with


In [104]:
vectors = pd.read_csv(open('vectors.tsv'), "\t", header=None)
vectors.head(10)

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.104471,0.123724,-0.025642,0.170934,-0.170223,-0.060754,-0.043021,-0.164961,-0.016186,0.016860,...,-0.145533,-0.079968,0.061998,-0.062237,-0.029635,0.092744,0.210847,-0.036280,-0.011380,0.003587
1,-0.006792,-0.110834,0.045501,0.398507,0.396435,-0.075430,0.324890,-0.114088,-0.123256,0.173011,...,0.059177,0.136777,-0.013857,0.178122,-0.285843,0.338755,0.249153,0.252935,-0.084933,0.065256
2,0.066751,0.267606,0.126228,0.170005,-0.028708,-0.135948,0.113617,-0.628945,0.033602,-0.047891,...,-0.238976,0.133630,0.006304,-0.531253,-0.437730,-0.051536,-0.016540,0.236779,-0.143301,-0.150719
3,-0.163334,0.360898,-0.063142,0.109465,-0.177643,-0.049343,-0.158552,-0.311239,0.438969,-0.016598,...,-0.415205,0.217786,0.072768,0.142416,-0.267845,0.517567,-0.404965,0.132778,-0.014412,0.084391
4,-0.020489,-0.188363,0.036605,0.282989,0.191124,-0.115724,0.350659,0.037417,-0.293113,0.478307,...,0.211098,0.536225,-0.101885,0.135577,-0.288659,0.495842,0.106861,0.258738,-0.057103,0.272932
5,-0.273582,0.479505,-0.141541,0.254470,0.023591,-0.230999,0.186081,-0.192405,-0.426040,-0.024954,...,0.109690,0.243635,0.213520,-0.157307,-0.146685,-0.165633,0.040485,-0.298309,-0.165945,-0.229455
6,-0.060743,-0.129491,-0.234270,0.122797,0.183565,-0.270347,0.112958,-0.243268,-0.025924,0.125528,...,-0.457792,0.143126,0.134445,-0.312558,-0.126989,0.497806,-0.233411,0.016862,0.038610,-0.219572
7,0.004355,0.167091,-0.244875,0.076883,-0.210852,0.360729,-0.203395,-0.088500,-0.082860,-0.150549,...,-0.333093,-0.050630,-0.003419,-0.050767,-0.485347,0.276742,0.424615,0.254225,-0.198032,-0.010091
8,0.170196,0.165078,-0.264543,0.280278,-0.028384,0.034147,0.165601,-0.054884,-0.122320,-0.186876,...,-0.233746,0.420095,0.390636,-0.261004,0.201526,-0.027658,-0.111250,0.070305,-0.027758,-0.406739
9,0.045645,-0.092531,0.048713,0.368786,-0.092174,-0.191822,-0.128511,-0.415022,-0.419463,0.152646,...,0.317856,-0.083330,0.373449,-0.138778,-0.336067,-0.330186,-0.097870,0.082105,-0.261599,-0.563813


In [107]:
import io

In [153]:
vector_len = len(vectors.columns)

In [154]:
# creates a file that stores the X array, vectors of word embeddings
def vectorize_synopses(X_indices, filename):
    
    vectorized_synopses = []

    for synopsis in X_indices:

        v_synopsis = []

        for word_index in synopsis:

            vector_index = word_index-1 # metadata and index2word indices offset by one

            if vector_index != -1:
                v_synopsis.append(vectors.loc[vector_index]) # represent every word in synopsis with a vector
            else:
                v_synopsis.append([0]*vector_len) # zero padding vector
        
        vectorized_synopses.append(v_synopsis)
    
    with open(filename, 'wb') as f:
        np.save(f, vectorized_synopses)


In [155]:
vectorize_synopses(X_train, 'x_train.npy')

In [ ]:
vectorize_synopses(X_test, 'x_test.npy')

In [ ]:
vectorize_synopses(X_valid, 'x_valid.npy')

In [ ]:
with open('x_train.npy', 'rb') as f:
    x_train = np.load(f)

Error: Session cannot generate requests

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.layers import Flatten, Conv1D, MaxPool1D
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import BinaryCrossentropy


In [ ]:
model = Sequential()

nb_filters=16
filter_size=3
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal', input_shape=(seq_length, 1)))
model.add(MaxPool1D()) 
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal'))
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal'))
model.add(MaxPool1D()) 
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal'))
model.add(MaxPool1D()) 
model.add(Flatten())
model.add(Dense(1, activation='linear'))

In [ ]:
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_loss', 
                   patience=50, verbose=1,
                   restore_best_weights=True)



In [ ]:
import tensorflow_datasets as tfds

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [ ]:
x_train = []
for x in tfds.as_numpy(X_train):
    x_train.append(x)





TypeError: Arguments to as_numpy must be tf.Tensors or tf.data.Datasets. Got: <class 'numpy.ndarray'>.

In [ ]:
# what is going on?
# Error Message: 
"""/Users/lanearohan/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: 
VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of 
lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. 
If you meant to do this, you must specify 'dtype=object' when creating the ndarray
#Entry point for launching an IPython kernel."""

X_train = np.array(x_train)

/Users/lanearohan/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:

#Y_train.size()

NameError: name 'count' is not defined

In [ ]:
# size issues between X_train and Y_train ... ????????

model.fit(x_train, Y_train, batch_size=32, validation_split=0.2, callbacks=[es], epochs=100)

ValueError: Data cardinality is ambiguous:
  x sizes: 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 504, 5

In [ ]:
preds = model.predict(X_test)

2021-11-18 21:26:10.570553: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
